In [1]:
!pip install -U langchain langchain-community langchain-google-genai tiktoken sentence-transformers

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_google_genai-2.1.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached grpcio_status-1.73.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.me

In [2]:
!pip install -U pinecone

  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached pinecone_plugin_assistant-1.7.0-py3-none-any.whl.metadata (28 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)
Using cached pinecone_plugin_assistant-1.7.0-py3-none-any.whl (239 kB)

   ------------- -------------------------- 1/3 [pinecone-plugin-assistant]
   ------------- -------------------------- 1/3 [pinecone-plugin-assistant]
   -------------------------- ------------- 2/3 [pinecone]
   -------------------------- ------------- 2/3 [pinecone]
   -------------------------- ------------- 2/3 [pinecone]
   -------------------------- ------------- 2/3 [pinecone]
   -------------------------- ------------- 2/3 [pinecone]
   -------------------------- ------------- 2/3 [pinecone]
   ---------------------------------------- 3/3 [pinecone]



In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
import os

In [7]:
loader=TextLoader("input.txt")
documents=loader.load()

In [8]:
splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=splitter.split_documents(documents)

Created a chunk of size 673, which is longer than the specified 500
Created a chunk of size 512, which is longer than the specified 500
Created a chunk of size 684, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 709, which is longer than the specified 500


In [19]:
import os
# Set environment variables
os.environ["GOOGLE_API_KEY"] = "your_gemini_api_key"
os.environ["PINECONE_API_KEY"] = "your_pinecone_api_key"

# Access environment variables
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [30]:
from pinecone import Pinecone,ServerlessSpec
# api_key="pcsk_3RTaU4_J9pCdVHPP9ThxVBkDbivYHKdue77UEk2vomD7HUtgHp7gg6EnGGsceuA6p7iGzm"
environment="us-east-1"
index_name="gemini-chat-index"
pc=Pinecone(api_key=PINECONE_API_KEY)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1"),
    )
index=pc.Index(index_name)

In [26]:
!pip install langchain_pinecone

   ---------------------------------------- 0.0/755.6 kB ? eta -:--:--
   ---------------------------------------- 755.6/755.6 kB 8.4 MB/s eta 0:00:00

   --- ------------------------------------  1/13 [vcrpy]
   ------ ---------------------------------  2/13 [syrupy]
   ------------ ---------------------------  4/13 [pytest-benchmark]
   ------------------ ---------------------  6/13 [pytest-recording]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   ------------------------ ---------------  8/13 [openai]
   

In [31]:
import os
import pinecone  # Import pinecone

# Connect to Pinecone (v7.3.0 syntax)
pc = pinecone.Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# For LangChain compatibility with Pinecone v2 API
from langchain_pinecone import PineconeVectorStore  # Use the updated import

# Create the vector store
vectorstore = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=index_name,
)

In [39]:
# Step 1: Set up Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

In [42]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

In [43]:
# Step 3: Ask your question
query = "Is universe has edge or any corner"
result = qa_chain.invoke({"query": query})

# Step 4: Show answer
print("🔎 Answer:\n", result['result'])

# (Optional) Show sources
for i, doc in enumerate(result['source_documents']):
    print(f"\n--- Source #{i+1} ---\n{doc.page_content}")

🔎 Answer:
 Based on the provided text, the universe has neither an edge nor a center.  At the largest scales, galaxies are distributed uniformly in all directions.

--- Source #1 ---
The universe is all of space and time[a] and their contents.[9] It comprises all of existence, any fundamental interaction, physical process and physical constant, and therefore all forms of matter and energy, and the structures they form, from sub-atomic particles to entire galactic filaments. Since the early 20th century, the field of cosmology establishes that space and time emerged together at the Big Bang 13.787Â±0.020 billion years ago[10] and that the universe has been expanding since then. The portion of the universe that can be seen by humans is approximately 93 billion light-years in diameter at present, but the total size of the universe is not known.[3]

--- Source #2 ---
Further observational improvements led to the realization that the Sun is one of a few hundred billion stars in the Milky Wa